## Examine NARS table data | [NARS](https://www.epa.gov/national-aquatic-resource-surveys) | [StreamCat](https://www.epa.gov/national-aquatic-resource-surveys/streamcat)

In [ ]:
import os
import pandas as pd
folder = 'data/nars'
for f in os.listdir('./%s' % folder):  # loop through every file in a given directory
    if f.split('.')[-1] == 'csv':  # find only CSV's
        print ('*************************')
        print (f)
        if f.split('.')[0] == 'wsa_siteinfo_ts_final':  # load NARS site info into a pandas DataFrame
            sites = pd.read_csv('{}/{}'.format(folder,f))

In [ ]:
sites.head()  # look at the first 5 records
# sites.columns

### * get water chemistry data to merge w/ LAT/LON in site info

In [ ]:
chem = pd.read_csv('{}/waterchemistry.csv'.format(folder))
print(chem.columns) #.tolist()
chem = chem.loc[(chem.SAMPLED == 'Yes'),:]

In [ ]:
print(pd.options.display.max_columns) # default -- 20
print(pd.options.display.max_rows) # default -- 60

In [ ]:
# pd.options.display.max_rows = 47
# chem = chem [['SITE_ID','NO3', 'NH4', 'NTL']]
sites.drop_duplicates('SITE_ID', inplace=True)
sitechem = pd.merge(sites, chem, on='SITE_ID')
sitechem.head()

In [ ]:
print(len(sites))
print(len(chem))
print(len(sitechem))

In [ ]:
% matplotlib notebook 
sitechem['EPAREGION'].value_counts().plot(kind='bar', title="How many NARS sites in each EPA REGION?", figsize=(4,3), rot=47)

#### What is an [EPA REGION](https://www.epa.gov/aboutepa/visiting-regional-office)??

In [ ]:
sitechem[['LON_DD','LAT_DD']].head()  # use these columns for point geometry

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(sitechem.LON_DD, sitechem.LAT_DD)]  # list comprehension to create geometries
crs = {'init': 'epsg:4326'}  # initialize the CRS with a dictionary
gdf = gpd.GeoDataFrame(sitechem, crs=crs, geometry=geometry)  

In [ ]:
gdf.loc[:5,:].plot()

In [ ]:
print(gdf.unary_union.bounds) # Another way to get to the center of your data
extent = gdf.unary_union.bounds
lat_Center = (extent[1] + extent[3]) / 2
lon_Center = (extent[0] + extent[2]) / 2

In [ ]:
import folium
from folium import plugins

nars_map = folium.Map(location=[lat_Center, lon_Center], 
                        tiles      = ("https://api.mapbox.com/styles/v1/mapbox/"
                                       "streets-v10/tiles/256/{z}/{x}/{y}?access_token="
                                       "pk.eyJ1IjoiZGFyYW9iZWlybmUiLCJhIjoidlV2WXZKRSJ9.EwVOY6b8UeyIz3N2UAu5pw"),
                        zoom_start = 4,
                        attr       = '© Mapbox')
marker_cluster = plugins.MarkerCluster().add_to(nars_map)

# for name, row in sitechem.iterrows():
for name, row in sitechem.ix[:470].iterrows():  # loop through every row to add each point to map w/ a popup
    x, y = row['LON_DD'],row['LAT_DD']
    
    html="""
        <style>
            h4 { 
              color: white;
              background-color: #303030;
              text-align:center;
              margin-bottom:0;
            }
            table {
              border: "1";
              margin: 0px auto;
              width: 283px;
            }
            tr {
              background-color:grey;
              text-align: center;
            }
            .nitrate {
              background-color:yellow;
            }             
        </style>
        <h4>SITE_ID: %s</h4>
        <table>
          <tr class="nitrate">
              <td>Nitrate (ueq/L): </td>
              <td>%s</td>
          </tr>
          <tr>
              <td>Ammonium (ueq/L): </td>
              <td>%s</td>
          </tr>          
          <tr>
              <td>Total Nitrogen (ug/L): </td>
              <td>%s</td>
          </tr>      
        </table>
    """ % (row.SITE_ID, row.NO3, row.NH4, row.NTL)
     
    iframe = folium.IFrame(html=html, width=300, height=120)
    folium.Marker([y, x], popup=folium.Popup(iframe)).add_to(marker_cluster)


# nars_map.save('maps/nars_map.html')  #saves nars_map.html in the working directory
nars_map

### What is this?

In [ ]:
import this